# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [1]:
# Load the libraries as required.
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

In [2]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


# Get X and Y

Create the features data frame and target data.

In [3]:
target_col = "area"          # the variable we want to predict
X = fires_dt.drop(columns=[target_col])
print(f"Shape of X  : {X.shape}")

Shape of X  : (517, 12)


In [4]:
y = fires_dt[target_col]
print(f"Shape of y  : {y.shape}")
X.head()

Shape of y  : (517,)


,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0


# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [29]:
num_cols = X.select_dtypes(include=["number"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object"]).columns.tolist()

# Preprocessing 1: scaling + one-hot encoding
preproc1 = ColumnTransformer([
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse=False), cat_cols)
])

### Preproc 2

Create preproc2 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [32]:
from sklearn.preprocessing import PowerTransformer


num_pipe = Pipeline([
    ("power", PowerTransformer(method="yeo-johnson", standardize=False)),
    ("scale", StandardScaler())
])

# categorical pipeline: one‑hot encode
cat_pipe = OneHotEncoder(handle_unknown="ignore", sparse=False)

# column transformer
preproc2 = ColumnTransformer([
    ("num", num_pipe, num_cols),
    ("cat", cat_pipe, cat_cols)
])

# quick check
Xt2 = preproc2.fit_transform(X)
print("Preproc2 transformed shape:", Xt2.shape)

Preproc2 transformed shape: (517, 29)


c:\Users\vrush\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [ ]:
from sklearn.model_selection import train_test_split

#  X is our feature matrix and y is our target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set size:", X_train.shape)
print("Test set size:", X_test.shape)

Training set size: (413, 12)
Test set size: (104, 12)


In [33]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [34]:
# Pipeline A = preproc1 + baseline
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

pipeline_a = Pipeline([
    ("preprocessing", preproc1),
    ("regressor", LinearRegression())
])

# Fit and evaluate
pipeline_a.fit(X_train, y_train)
score_a = pipeline_a.score(X_test, y_test)
print("Pipeline A Test Score:", score_a)
pipeline_a

Pipeline A Test Score: 0.014768502010551687


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['month', 'day'])])),
                ('regressor', LinearRegression())])

In [35]:
# Pipeline B = preproc2 + baseline

pipeline_b = Pipeline([
    ("preprocessing", preproc2),
    ("regressor", LinearRegression())
])

# Fit and evaluate
pipeline_b.fit(X_train, y_train)
score_b = pipeline_b.score(X_test, y_test)
print("Pipeline B Test Score:", score_b)
pipeline_b


Pipeline B Test Score: 0.018604020051235293


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('power',
                                                                   PowerTransformer(standardize=False)),
                                                                  ('scale',
                                                                   StandardScaler())]),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['month', 'day'])])),
                ('regressor', LinearRegression())])

In [36]:
# Pipeline C = preproc1 + advanced model

from sklearn.ensemble import RandomForestRegressor

pipeline_c = Pipeline([
    ("preprocessing", preproc1),
    ("regressor", RandomForestRegressor(n_estimators=100))
])

# Fit and evaluate
pipeline_c.fit(X_train, y_train)
score_c = pipeline_c.score(X_test, y_test)
print("Pipeline C Test Score:", score_c)
pipeline_c

Pipeline C Test Score: 0.0038972212574498455


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['month', 'day'])])),
                ('regressor', RandomForestRegressor())])

In [37]:
# Pipeline D = preproc2 + advanced model

pipeline_d = Pipeline([
    ("preprocessing", preproc2),
    ("regressor", RandomForestRegressor(n_estimators=100))
])

# Fit and evaluate
pipeline_d.fit(X_train, y_train)
score_d = pipeline_d.score(X_test, y_test)
print("Pipeline D Test Score:", score_d)
pipeline_d    

Pipeline D Test Score: -0.007293019455377747


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('power',
                                                                   PowerTransformer(standardize=False)),
                                                                  ('scale',
                                                                   StandardScaler())]),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['month', 'day'])])),
                ('regressor', RandomForestRegressor())])

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [ ]:
# hyperparameter grid for Pipeline A (LinearRegression)
param_grid_a = {
    "regressor__fit_intercept": [True, False]
}

grid_a = GridSearchCV(pipeline_a, param_grid_a, cv=5, scoring="r2")
grid_a.fit(X_train, y_train)

print("Best hyperparameters for Pipeline A:", grid_a.best_params_)
print("Best score for Pipeline A:", grid_a.best_score_)


Best hyperparameters for Pipeline A: {'regressor__fit_intercept': False}
Best score for Pipeline A: -0.31526951962422445


In [ ]:
# hyperparameter grid for Pipeline B (LinearRegression)
param_grid_b = {
    "regressor__fit_intercept": [True, False]
}

grid_b = GridSearchCV(pipeline_b, param_grid_b, cv=5, scoring="r2")
grid_b.fit(X_train, y_train)

print("Best hyperparameters for Pipeline B:", grid_b.best_params_)
print("Best score for Pipeline B:", grid_b.best_score_)



Best hyperparameters for Pipeline B: {'regressor__fit_intercept': False}
Best score for Pipeline B: -0.24571069719190525


In [ ]:
# hyperparameter grid for Pipeline C (RandomForestRegressor)
param_grid_c = {
    "regressor__n_estimators": [50, 100, 200, 300],  # Number of trees
    "regressor__max_depth": [None, 10, 20, 30]  # Tree depth
}

grid_c = GridSearchCV(pipeline_c, param_grid_c, cv=5, scoring="r2")
grid_c.fit(X_train, y_train)

print("Best hyperparameters for Pipeline C:", grid_c.best_params_)
print("Best score for Pipeline C:", grid_c.best_score_)


Best hyperparameters for Pipeline C: {'regressor__max_depth': 20, 'regressor__n_estimators': 100}
Best score for Pipeline C: -1.2767138152312554


In [41]:
param_grid_d = {
    "regressor__n_estimators": [50, 100, 200, 300],  # Number of trees
    "regressor__max_depth": [None, 10, 20, 30]  # Tree depth
}

grid_d = GridSearchCV(pipeline_d, param_grid_d, cv=5, scoring="r2")
grid_d.fit(X_train, y_train)

print("Best hyperparameters for Pipeline D:", grid_d.best_params_)
print("Best score for Pipeline D:", grid_d.best_score_)

Best hyperparameters for Pipeline D: {'regressor__max_depth': 10, 'regressor__n_estimators': 50}
Best score for Pipeline D: -1.1897908555864922


# Evaluate

+ Which model has the best performance?

In [47]:
# Evaluate models on the test set
test_scores = {
    "Pipeline A ": grid_a.best_estimator_.score(X_test, y_test),
    "Pipeline B ": grid_b.best_estimator_.score(X_test, y_test),
    "Pipeline C ": grid_c.best_estimator_.score(X_test, y_test),
    "Pipeline D ": grid_d.best_estimator_.score(X_test, y_test)
}

# Print results
for pipeline, score in test_scores.items():
    print(f"{pipeline}: Test Score = {score:.4f}")


Pipeline A : Test Score = 0.0148
Pipeline B : Test Score = 0.0186
Pipeline C : Test Score = 0.0050
Pipeline D : Test Score = -0.0141


# Export

+ Save the best performing model to a pickle file.

In [48]:
import pickle

best_model = grid_d.best_estimator_  

# Save to a pickle file
with open("best_model_forest.pkl", "wb") as file:
    pickle.dump(best_model, file)

print("Best model saved successfully as best_model_forest.pkl")

Best model saved successfully as best_model_forest.pkl


# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [87]:
print(X_test.dtypes)

coord_x      int64
coord_y      int64
month       object
day         object
ffmc       float64
dmc        float64
dc         float64
isi        float64
temp       float64
rh           int64
wind       float64
rain       float64
dtype: object


In [73]:
#X_test_cleaned = X_test.copy()

# Convert categorical features to numeric encoding (ensuring dtype consistency)
#X_test_cleaned["month"] = X_test_cleaned["month"].astype("category").cat.codes
#X_test_cleaned["day"] = X_test_cleaned["day"].astype("category").cat.codes

#print(X_test_cleaned.dtypes)  # Verify everything is numeric


In [81]:
#import numpy as np

# Ensure all data types are compatible with NumPy
#X_test_cleaned = X_test_cleaned.astype(np.float64)  # Convert entire dataset to float64

#print("Final Data Types:", X_test_cleaned.dtypes)  # Ensure all columns are float or int

In [85]:
import numpy as np
np.int = int

In [95]:
print(X_test_cleaned.dtypes)  # Confirm all columns are numeric
print(X_test_cleaned.head())  # Check for strange values
X_test_cleaned = X_test_cleaned.apply(pd.to_numeric, errors='coerce')
import numpy as np

# Replace non-finite values
X_test_cleaned = X_test_cleaned.replace([np.inf, -np.inf], np.nan).dropna()

print("Final dataset shape:", X_test_cleaned.shape)  # Ensure no missing values
X_test_cleaned_array = X_test_cleaned.values.astype(np.float64)  # Ensure correct dtype



coord_x    float64
coord_y    float64
month      float64
day        float64
ffmc       float64
dmc        float64
dc         float64
isi        float64
temp       float64
rh         float64
wind       float64
rain       float64
dtype: object
     coord_x  coord_y  month  day  ffmc    dmc     dc   isi  temp    rh  wind  \
304      6.0      5.0    8.0  2.0  85.1   28.0  113.8   3.5  11.3  94.0   4.9   
501      7.0      5.0    1.0  5.0  96.1  181.1  671.2  14.3  21.6  65.0   4.9   
441      8.0      6.0    1.0  1.0  92.1  207.0  672.6   8.2  25.5  29.0   1.8   
153      5.0      4.0   10.0  0.0  94.3   85.1  692.3  15.9  20.1  47.0   4.9   
503      2.0      4.0    1.0  6.0  94.5  139.4  689.1  20.0  29.2  30.0   4.9   

     rain  
304   0.0  
501   0.8  
441   0.0  
153   0.0  
503   0.0  
Final dataset shape: (104, 12)


In [96]:
#explainer = shap.Explainer(best_model.predict, X_test_cleaned_array)
#shap_values = explainer(X_test_cleaned_array)
#shap.summary_plot(shap_values, X_test_cleaned)


In [97]:
#subset = X_test_cleaned.iloc[:100]  # First 100 rows
#explainer = shap.Explainer(best_model.predict, subset)
#shap_values = explainer(subset)

*(Answer here.)*

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.